# Train Distributed Model

In [2]:
import tensorflow as tf
# Server 0:
cluster = tf.train.ClusterSpec({"local": ["localhost:2222", "localhost:2223"]})


UnknownError: Could not start gRPC server

In [3]:
server0 = tf.train.Server(cluster, job_name="local", task_index=0)

print(server0)

In [4]:
server1 = tf.train.Server(cluster, job_name="local", task_index=1)

print(server1)

In [ ]:
import tensorflow as tf
import datetime
import numpy as np

n = 2
c1 = tf.Variable([])
c2 = tf.Variable([])

def matpow(M, n):
    if n < 1: 
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))

# Assigns ops to the local worker by default.
#with tf.device(tf.train.replica_device_setter(
#        worker_device="/job:worker/task:%d" % FLAGS.task_index,
#        cluster=cluster)):

with tf.device("/job:local/task:0/cpu:0"):
    A = tf.random_normal(shape=[100, 100])
    #np.random.rand(1e2, 1e2)
    #.astype('float32')
    c1 = matpow(A,n)

with tf.device("/job:local/task:1/cpu:0"):
    B = tf.random_normal(shape=[100, 100])
    #np.random.rand(1e2, 1e2)
    #.astype('float32')
    c2 = matpow(B,n)

with tf.Session("grpc://127.0.0.1:2222") as sess:
    t1 = datetime.datetime.now()
    sum = c1 + c2
    t2 = datetime.datetime.now()
    print(sess.run(sum))
    print("Multi node computation time: " + str(t2-t1))